# Visualisatie landelijk beeld
Run het notebook cell voor cell. In de eerste cell kun je het `.xls`-bestand wat uit Hytdra-NL komt uploaden, waarna je in de tweede cell de resultaten kan analyseren voor verschillende terugkeertijden.

In [7]:
import pandas as pd
import geopandas as gpd

from ipywidgets import FileUpload, Label

label = Label(value="Upload een .xls bestand met de berekeningen.")
uploader = FileUpload(
    accept='*.xls',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

gdf_berekeningen = None

def load_data(file):
    global gdf_berekeningen
    if uploader.value:
        uploaded_file = uploader.value[0]
        content = uploaded_file.content
        df_berekeningen = pd.read_csv(pd.io.common.BytesIO(content), sep="\t", encoding="ansi", decimal=",")
        gdf_berekeningen = gpd.GeoDataFrame(
            df_berekeningen,
            geometry=gpd.points_from_xy(df_berekeningen["X-coördinaat"], df_berekeningen["Y-coördinaat"]),
            crs="EPSG:28992"
        )
    label.value = f"Bestand '{uploaded_file.name}' is geüpload."

uploader.observe(load_data, names='value')

display(uploader, label)

# Inlezen van de keringlijnen
gdf_keringlijnen = gpd.read_file(r"C:\Users\tolp2\Documents\projects\Verschilanalyse BOI - visualisatie\NBPW\voorbeeldbestand_nationaalBestandPrimaireWaterkeringen.shp").to_crs("EPSG:28992")

FileUpload(value=(), accept='*.xls', description='Upload')

Label(value='Upload een .xls bestand met de berekeningen.')

In [8]:
from ipywidgets import interact
import contextily as cx
from matplotlib import cm
import matplotlib.pyplot as plt
from shapely.geometry import box

def update_plot(terugkeertijd):
    fig = plt.figure(
        layout="constrained",
        figsize=(20, 10),
        dpi=100
    )
    axd = fig.subplot_mosaic(
        [
            ["map", "histo1"],
            ["map", "histo2"],
        ],
        width_ratios=[2, 1],
    )

    fig.suptitle(f"Bovenrijn - waterstanden voor terugkeertijd: {terugkeertijd} jaar", fontsize=16)

    # Plot the map
    gdf_berekeningen_filtered = gdf_berekeningen[gdf_berekeningen['Terugkeertijd [jaar]'] == terugkeertijd]
    # - laat de hoogte van de punten zien als kleur, met een aangepaste colormap
    gdf_berekeningen_filtered.to_crs(epsg=3857).plot(ax=axd["map"], column='Belastingniveau [m+NAP]/Golfparameter [m]/[s]/Sterkte bekleding [-]', cmap=cm.Spectral_r, alpha=0.85, edgecolor="k")
    # - voeg een achtergrondkaart toe
    cx.add_basemap(axd["map"], source=cx.providers.CartoDB.Positron)
    axd["map"].set_axis_off()
    plt.colorbar(axd["map"].get_children()[0])
    #  - zoom op het gebied van de punten
    minx, miny, maxx, maxy = gdf_berekeningen_filtered.total_bounds
    buffered_bbox = box( minx, miny, maxx, maxy ).buffer(10000)  # Buffer van 5000 meter
    gdf_keringlijnen.intersection(buffered_bbox).to_crs(epsg=3857).plot(ax=axd["map"], color='blue', linewidth=0.5)

    # Print een histogram
    gdf_berekeningen_filtered['Belastingniveau [m+NAP]/Golfparameter [m]/[s]/Sterkte bekleding [-]'].plot.hist(ax=axd["histo1"], bins=20)
    axd["histo1"].set_title("Histogram van waterstand")
    axd["histo1"].set_xlabel("Waterstand [m+NAP]")

    # Print een overschrijdingslijn


interact(update_plot, terugkeertijd=gdf_berekeningen['Terugkeertijd [jaar]'].unique());

interactive(children=(Dropdown(description='terugkeertijd', options=(np.float64(10.0), np.float64(30.0), np.fl…